In [1]:
from sklearn import svm
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import keras
from sklearn import svm
from sklearn import metrics

In [2]:
import myModelDispConv

new_modelRaw = myModelDispConv.load_model()

In [3]:
new_modelRaw.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data_unnormalized (InputLayer)  [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
data_ (Normalization)           (None, 227, 227, 3)  309175      data_unnormalized[0][0]          
__________________________________________________________________________________________________
conv1_ (Conv2D)                 (None, 113, 113, 64) 1792        data_[0][0]                      
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 113, 113, 64) 0           conv1_[0][0]                     
______________________________________________________________________________________________

In [4]:
def sample_detach_feature (imgs_dir, labels_dir, features):
    
    X = np.load(imgs_dir)
    Y = np.load(labels_dir)
    feat = np.load(features)

    #divide os dados em 80% para treino e 20% para teste
    train_X, test_X, train_label, test_label = train_test_split(X, Y, test_size=0.2, random_state=13)

    #divide os dados de treino em 80% para treino e 20% para validação
    train_X, valid_X, train_label, valid_label = train_test_split(train_X, train_label, test_size=0.2, random_state=13)

    #divide os dados em 80% para treino e 20% para teste
    train_feat_X, test_feat_X, train_label, test_label = train_test_split(feat, Y, test_size=0.2, random_state=13)

    #divide os dados de treino em 80% para treino e 20% para validação
    train_feat_X, valid_feat_X, train_label, valid_label = train_test_split(train_feat_X, train_label, test_size=0.2, random_state=13)

    #mostra os shapes resultantes das divisões
    print("Training, test and validation shape")
    print("Training:   ", train_X.shape, train_label.shape)
    print("Validation: ", valid_X.shape, valid_label.shape)
    print("Testing:    ", test_X.shape, test_label.shape)

    return train_X, train_label, test_X, test_label, valid_X, valid_label, train_feat_X, test_feat_X, valid_feat_X

In [10]:
def predict_SVM (model, layerName, train_X, train_label, test_X, test_label, feature_train, feature_test, kernelType = 'rbf'):
    
    intermediate_layer_model = create_intermediate_model(model, layerName)
    
    intermediate_output = intermediate_layer_model.predict(train_X)
    print("INTERMEDIATE OUTPUT ", intermediate_output.shape)
    
    intermediate_output = np.reshape(intermediate_output, (intermediate_output.shape[0], -1))
    print("INTERMEDIATE shape: ",intermediate_output.shape)
    print("FEATURE TRAIN" ,feature_train.shape)
    
    all_features = np.append(intermediate_output, feature_train, axis=1)
    
    print("Features shape: ",all_features.shape)
    clf = svm.SVC(kernel = kernelType)
    predicted_classes = np.argmax(train_label, axis=1) #no onehot
    clf.fit(all_features, predicted_classes)
    
    intermediate_output_test = intermediate_layer_model.predict(test_X)
    intermediate_output_test = np.reshape(intermediate_output_test, (intermediate_output_test.shape[0], -1))
    
    all_features_test = np.append(intermediate_output_test, feature_test, axis=1)
    
    y_pred=clf.predict(all_features_test)
    predicted_classes = np.argmax(test_label, axis=1)
    print("Accuracy SVM:",metrics.accuracy_score(predicted_classes, y_pred))

In [11]:
def create_intermediate_model (model, layerName):
    intermediate_layer_model = keras.Model(inputs=model.input,
                                       outputs=model.get_layer(layerName).output)
    return intermediate_layer_model

In [12]:
img_disp = 'E:/MODELS_DISPLASIA_BACKUP/MODELS_DISPLASIA/images_np/all_imgs_disp_227/all_imgs_disp_227.npy'
label_disp = 'E:/MODELS_DISPLASIA_BACKUP/MODELS_DISPLASIA/labels/disp.npy'
feat_disp_den = 'E:/MODELS_DISPLASIA_BACKUP/MODELS_DISPLASIA/all_flatten_features/all_5_features_disp_conv.npy'

In [13]:
train_X, train_label, test_X, test_label, valid_X, valid_label, train_feat_X, test_feat_X, valid_feat_X = sample_detach_feature(img_disp, label_disp, feat_disp_den)

Training, test and validation shape
Training:    (8124, 227, 227, 3) (8124, 4)
Validation:  (2032, 227, 227, 3) (2032, 4)
Testing:     (2540, 227, 227, 3) (2540, 4)


In [14]:
predict_SVM(new_modelRaw, 'global_average_pooling2d', train_X, train_label, test_X, test_label, train_feat_X, test_feat_X)

INTERMEDIATE OUTPUT  (8124, 1, 1, 1000)
INTERMEDIATE shape:  (8124, 1000)
FEATURE TRAIN (8124, 5)
Features shape:  (8124, 1005)
Accuracy SVM: 0.7177165354330709
